## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP1 - Preparación de los datos

---

En esta libreta se realizan las transforamciones sobre los datasets del escenario en función 
de los resultados de la verificación de calidad de datos. 

---

In [2]:
#Se importan las librerias a utilizar

import pandas as pd

----

##### Lectura de los datasets

In [20]:
df_creditos = pd.read_csv("../data/processed/datos_creditos_c.csv", sep=";")
display(df_creditos.head(1))

df_tarjetas = pd.read_csv("../data/processed/datos_tarjetas_c.csv", sep=";")
display(df_tarjetas.head(1))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,713061558.0,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,Blue,24.0,2.0


---
#### Aplicación de transformaciones

**Operaciones a realizar**

1. Selección de columnas
2. Filtrado de filas
3. Construcción de atributos
4. Integración de datasets
5. Formateo definitivo


----

Selección de datos

In [11]:
import numpy as np

def eliminar_col_correlacionadas(df, umbral=0.9, verbose=True):
    """
    Elimina una de dos columnas numéricas cuya correlación supere el umbral indicado.
    
    Parámetros:
        df (pd.DataFrame): DataFrame de entrada.
        umbral (float): Umbral de correlación para eliminar columnas.
        verbose (bool): Si es True, imprime las columnas eliminadas.
    
    Devuelve:
        pd.DataFrame: DataFrame con las columnas eliminadas.
    """
    df_numerico = df.select_dtypes(include=[np.number])  # Solo columnas numéricas
    matriz_correlacion = df_numerico.corr().abs()
    
    # Se considera solo la parte superior de la matriz de correlación (triángulo superior)
    matriz_superior = matriz_correlacion.where(np.triu(np.ones(matriz_correlacion.shape), k=1).astype(bool))
    
    # Se seleccionan las columnas que tienen al menos una correlación superior al umbral
    columnas_a_eliminar = [col for col in matriz_superior.columns if any(matriz_superior[col] > umbral)]
    
    if verbose and columnas_a_eliminar:
        print(f"Columnas eliminadas por tener correlación > {umbral}: {columnas_a_eliminar}")
    elif verbose:
        print("No se han eliminado columnas por correlación.")
    
    df_filtrado = df.drop(columns=columnas_a_eliminar)
    
    return df_filtrado


In [21]:
df_creditos = eliminar_col_correlacionadas(df_creditos, umbral=0.7)
df_tarjetas = eliminar_col_correlacionadas(df_tarjetas, umbral=0.7)

No se han eliminado columnas por correlación.
Columnas eliminadas por tener correlación > 0.7: ['operaciones_ult_12m']


In [6]:
print("Vista del dataset de datos de créditos:")
display(df_creditos.head(1))

print("Vista del dataset de datos de tarjetas:")
display(df_tarjetas.head(1))

Vista del dataset de datos de créditos:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y


Vista del dataset de datos de tarjetas:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,personas_a_cargo
0,713061558.0,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,Blue,2.0


Limpieza de los datos (filtrado a nivel de filas)

In [22]:
#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    
    if pct_ingreso > 0.5 and ingresos <= 20000:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'


# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_creditos.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_creditos = pd.concat([df_creditos, regla_pct_ingresos], axis=1)
df_creditos.head(5)  

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,regla_pct_ingresos
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y,ok
1,768805383.0,21,1000,2,5.0,PROPIA,9600,EDUCACIÓN,0.10,11.14,0,N,ok
2,818770008.0,25,5500,3,1.0,HIPOTECA,9600,SALUD,0.57,12.87,1,N,err
3,713982108.0,23,35000,2,4.0,ALQUILER,65500,SALUD,0.53,15.23,1,N,ok
4,710821833.0,24,35000,4,8.0,ALQUILER,54400,SALUD,0.55,14.27,1,Y,ok


In [23]:
print("Filas con edad < 90:", (df_creditos['edad'] > 90).sum())
print("Filas con antigüedad_empleado > 50:", (df_creditos['antiguedad_empleado'] > 50).sum())
print("Filas con antigüedad_empleado no nula:", df_creditos['antiguedad_empleado'].isna().sum())

Filas con edad < 90: 4
Filas con antigüedad_empleado > 50: 2
Filas con antigüedad_empleado no nula: 337


In [ ]:
# Se filtran las filas con algún error detectado
print(f"Filas antes del filtro: {df_creditos.shape[0]}")

# Filtro combinado:
# - edad < 90
# - antigüedad < 50
# - antigüedad no nula
temp = df_creditos[
    ((df_creditos['edad'] < 90) | (df_creditos['antiguedad_empleado'] < 50)) &
    (df_creditos['antiguedad_empleado'].notna())
]

# Filtro adicional: cumplimiento de la regla de negocio
temp_c = temp[temp['regla_pct_ingresos'] == 'ok']

print(f"Filas después del filtro: {temp_c.shape[0]}")

Filas antes del filtro: 10127
Filas después del filtro: 9776


Integración de datos

In [25]:
df_integrado = pd.merge(temp_c, df_tarjetas, on='id_cliente', how='inner')
coincidencias = df_integrado.shape[0]

print(f"Filas del dataset integrado con los filtros realizados: {coincidencias}")

Filas del dataset integrado con los filtros realizados: 9776


In [26]:
print(f"Cantidad de columnas del dataset integrado: {df_integrado.shape[1]}")

Cantidad de columnas del dataset integrado: 22


In [11]:
df_integrado.head(1)

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,regla_pct_ingresos,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,operaciones_ult_12m,personas_a_cargo
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,...,ok,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,24.0,2.0


#### Transformación de atributos

Atributos nominales que se modifican los valores

In [27]:
# Columna: estado_civil
cambios_estado_civil = {
    'CASADO' : 'C',
    'SOLTERO' : 'S',
    'DESCONOCIDO' : 'N',
    'DIVORCIADO' : 'D',
}

estado_civil_N = df_integrado.loc[:, ('estado_civil')].map(cambios_estado_civil).rename('estado_civil_N')

# Columna: estado_credito
cambios_estado_credito = {
    0: 'P',
    1 : 'C',
}

estado_credito_N = df_integrado.loc[:, ('estado_credito')].map(cambios_estado_credito).rename('estado_credito_N')

# Sobre este resultado será necesario eliminar las columnas auxiliares

Atributos numéricos que se discretizan

In [28]:
df_integrado['antiguedad_empleado'].describe()

count    9776.000000
mean        3.942205
std         3.320499
min         0.000000
25%         2.000000
50%         4.000000
75%         6.000000
max       123.000000
Name: antiguedad_empleado, dtype: float64

In [30]:
# Antiguedad del empleado

etiquetas_a_e = ['menor_5', '5_a_10', 'mayor_10']
rangos_a_e = [0, 4, 10, 50]
valor_para_nan = 'NA'
antiguedad_empleados_N = pd.cut(df_integrado['antiguedad_empleado'], 
                                bins=rangos_a_e, 
                                labels=etiquetas_a_e,
                                right=False).cat.add_categories(valor_para_nan).fillna(valor_para_nan)

antiguedad_empleados_N.value_counts()

# edad

etiquetas_e = ['menor_25', '25_a_30']
rangos_e = [0, 24, 50]
edad_N = pd.cut(df_integrado['edad'], 
                                bins=rangos_e, 
                                labels=etiquetas_e)

edad_N.value_counts()

# pct_ingreso

etiquetas_p_i = ['hasta_20', '20_a_40', '40_a_60', 'mayor_60']
rangos_p_i = [0, 0.19, 0.39, 0.60, 0.99]
pct_ingreso_N = pd.cut(df_integrado['pct_ingreso'], 
                                bins=rangos_p_i, 
                                labels=etiquetas_p_i)

pct_ingreso_N.value_counts()

# ingresos

etiquetas_i = ['hasta_20k', '20k_a_50k', '50k_a_100k', 'mayor_100k']
rangos_i = [0, 19999, 49999, 99999, 999999]
ingresos_N = pd.cut(df_integrado['ingresos'], 
                                bins=rangos_i, 
                                labels=etiquetas_i)

ingresos_N.value_counts()

# tasa_interes

etiquetas_t_i = ['hasta_7p', '7p_a_15p', '15p_a_20p', 'mayor_20p']
rangos_t_i = [0, 6.99, 14.99, 19.99, 100]
tasa_interes_N = pd.cut(df_integrado['tasa_interes'], 
                                bins=rangos_t_i, 
                                labels=etiquetas_t_i)

tasa_interes_N.value_counts()

# antiguedad_cliente

etiquetas_a_c = ['menor_2y', '2y_a_4y', 'mayor_4y']
rangos_a_c = [0, 24, 48, 100]
antiguedad_cliente_N = pd.cut(df_integrado['antiguedad_cliente'], 
                                bins=rangos_a_c, 
                                labels=etiquetas_a_c)

antiguedad_cliente_N.value_counts()

# limite_credito_tc

etiquetas_l_tc = ['menor_3k', '3k_a_5k', '5k_a_10k', 'mayor_10k']
rangos_l_tc = [0, 2999, 4999, 9999, 100000]
limite_credito_tc_N = pd.cut(df_integrado['limite_credito_tc'], 
                                bins=rangos_l_tc, 
                                labels=etiquetas_l_tc)

limite_credito_tc_N.value_counts()

# gastos_ult_12m

etiquetas_g_u12 = ['menor_1k', '2k_a_4k', '4k_a_6k', '6k_a_8k', '8k_a_10k', 'mayor_10k']
rangos_g_u12 = [0, 999, 3999, 5999, 7999, 9999, 100000]
gastos_ult_12m_N = pd.cut(df_integrado['gastos_ult_12m'], 
                                bins=rangos_g_u12, 
                                labels=etiquetas_g_u12)

gastos_ult_12m_N.value_counts()


gastos_ult_12m
2k_a_4k      4917
4k_a_6k      3292
mayor_10k     742
6k_a_8k       351
8k_a_10k      320
menor_1k      154
Name: count, dtype: int64

In [31]:
df_integrado.columns

Index(['id_cliente', 'edad', 'importe_solicitado', 'duracion_credito',
       'antiguedad_empleado', 'situacion_vivienda', 'ingresos',
       'objetivo_credito', 'pct_ingreso', 'tasa_interes', 'estado_credito',
       'falta_pago', 'regla_pct_ingresos', 'antiguedad_cliente',
       'estado_civil', 'estado_cliente', 'gastos_ult_12m', 'genero',
       'limite_credito_tc', 'nivel_educativo', 'nivel_tarjeta',
       'personas_a_cargo'],
      dtype='object')

In [32]:
# Agregar en este listado otros atributos que pudieran discretizarse o transformarse
col_eliminar_final = [
              'estado_civil',
              'estado_credito',
              'antiguedad_empleado',
              'edad',
              'antiguedad_cliente', 
              'ingresos',
              'pct_ingreso', 
              'tasa_interes',
              'regla_pct_ingresos',
              'gastos_ult_12m', 
              'limite_credito_tc', 
              'id_cliente']

df_integrado.drop(col_eliminar_final, inplace=True, axis=1)

df_final = pd.concat([ gastos_ult_12m_N, 
                      limite_credito_tc_N, 
                      antiguedad_cliente_N, 
                      tasa_interes_N, 
                      ingresos_N, 
                      pct_ingreso_N, 
                      antiguedad_empleados_N, 
                      edad_N, 
                      estado_civil_N, 
                      estado_credito_N,
                      df_integrado], axis=1)
df_final.head(5)


,gastos_ult_12m,limite_credito_tc,antiguedad_cliente,tasa_interes,ingresos,pct_ingreso,antiguedad_empleado,edad,estado_civil_N,estado_credito_N,importe_solicitado,duracion_credito,situacion_vivienda,objetivo_credito,falta_pago,estado_cliente,genero,nivel_educativo,nivel_tarjeta,personas_a_cargo
0,2k_a_4k,3k_a_5k,2y_a_4y,15p_a_20p,50k_a_100k,40_a_60,NA,menor_25,C,C,35000,3,ALQUILER,PERSONAL,Y,ACTIVO,M,UNIVERSITARIO_COMPLETO,Blue,2.0
1,2k_a_4k,mayor_10k,2y_a_4y,7p_a_15p,hasta_20k,hasta_20,5_a_10,menor_25,C,P,1000,2,PROPIA,EDUCACIÓN,N,ACTIVO,M,SECUNDARIO_COMPLETO,Blue,3.0
2,2k_a_4k,3k_a_5k,2y_a_4y,15p_a_20p,50k_a_100k,40_a_60,5_a_10,menor_25,C,C,35000,2,ALQUILER,SALUD,N,ACTIVO,M,UNIVERSITARIO_COMPLETO,Blue,3.0
3,2k_a_4k,5k_a_10k,mayor_4y,7p_a_15p,50k_a_100k,40_a_60,5_a_10,menor_25,C,C,35000,4,ALQUILER,SALUD,Y,ACTIVO,M,DESCONOCIDO,Blue,1.0
4,2k_a_4k,3k_a_5k,2y_a_4y,7p_a_15p,hasta_20k,20_a_40,menor_5,menor_25,N,C,2500,2,PROPIA,INVERSIONES,N,ACTIVO,F,SECUNDARIO_COMPLETO,Blue,4.0


Exportación de metadatos

In [33]:
from ydata_profiling import ProfileReport

profile = ProfileReport(df_final, title="Reporte de preparación de datos - Dataset integrado")
profile.to_file("../docs/reporte_datos_integrados.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [34]:
print(f"Cantidad de columnas del dataset final: {df_final.shape[1]}")
print(f"Cantidad de filas del dataset final: {df_final.shape[0]}")

Cantidad de columnas del dataset final: 20
Cantidad de filas del dataset final: 9776


Exportación del dataset

In [35]:
df_final.to_csv("../data/final/datos_finales.csv", sep=';', index=False)